In [1]:
!pip install --upgrade pip
!pip install gdown

     |████████████████████████████████| 2.1 MB 805 kB/s 
  Attempting uninstall: pip
    Found existing installation: pip 20.1.1
    Uninstalling pip-20.1.1:
      Successfully uninstalled pip-20.1.1
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14775 sha256=0459599f00dd1621a134160064e287937009f0d5b796ba65ec02da4b02f8114c
  Stored in directory: /root/.cache/pip/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown


In [2]:
from keras.models import load_model

In [3]:
import os
if not (os.path.isfile('data.zip') and os.path.isfile('data.zip')):
    !rm -rf model_discovery
    import gdown
    gdown.download('https://drive.google.com/uc?id=1exw9vOYcb0fPVseleifK0m1VxbBCsBM_', output=None, quiet=False)
    !unzip -qq model_discovery.zip

Downloading...
From: https://drive.google.com/uc?id=1exw9vOYcb0fPVseleifK0m1VxbBCsBM_
To: /kaggle/working/model_discovery.zip
100%|██████████| 11.9k/11.9k [00:00<00:00, 4.15MB/s]


In [4]:
# import deepmatcher as dm
import itertools
import os
import pandas as pd
import model_discovery
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from model_discovery import utils, l2lsh
import re

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet


import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras 
from keras import backend as K
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from keras.layers import LSTM,Dense,Bidirectional,Input
from keras.models import Model
import torch
import transformers

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [5]:
datasets = ['../input/imbd-data-1/Movie_1.csv', '../input/imbd-data-1/Movie_2.csv', 
            '../input/financial-news-data-2/Financial_News_2.csv', '../input/financial-news-data-2/Financial_News_1.csv',
            '../input/tweets-data-4/Tweets_1.csv','../input/tweets-data-4/Tweets_2.csv',
           '../input/tweets-hatred-data-5/Tweets_hatred_5.csv']

In [6]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i.strip().lower().isalpha():
            final_text.append(i.strip().lower())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

def fast_encode(texts, tokenizer, chunk_size=256, maxlen=400):

    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
#     print(texts)
    for i in range(0, len(texts), chunk_size):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)



In [7]:
def build_model(transformer, max_len=400):
    
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=2e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

from tokenizers import BertWordPieceTokenizer
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased' , lower = True)
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True)
bert_model = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')

In [8]:
comb = list(itertools.permutations(datasets, 2))
print("Source,                                                    Target")
for c in comb:
    print(c[0],"                 ",c[1])

Source,                                                    Target
../input/imbd-data-1/Movie_1.csv                   ../input/imbd-data-1/Movie_2.csv
../input/imbd-data-1/Movie_1.csv                   ../input/financial-news-data-2/Financial_News_2.csv
../input/imbd-data-1/Movie_1.csv                   ../input/financial-news-data-2/Financial_News_1.csv
../input/imbd-data-1/Movie_1.csv                   ../input/tweets-data-4/Tweets_1.csv
../input/imbd-data-1/Movie_1.csv                   ../input/tweets-data-4/Tweets_2.csv
../input/imbd-data-1/Movie_1.csv                   ../input/tweets-hatred-data-5/Tweets_hatred_5.csv
../input/imbd-data-1/Movie_2.csv                   ../input/imbd-data-1/Movie_1.csv
../input/imbd-data-1/Movie_2.csv                   ../input/financial-news-data-2/Financial_News_2.csv
../input/imbd-data-1/Movie_2.csv                   ../input/financial-news-data-2/Financial_News_1.csv
../input/imbd-data-1/Movie_2.csv                   ../input/tweets-data-4/Tweet

In [9]:
arr_model_already_trained=[]
def model_serving(src, tar):
    src_name=src
    tar_name=tar
    src=pd.read_csv(src).sample(2000, replace=True,random_state=10)
    tar=pd.read_csv(tar).sample(2000, replace=True,random_state=10)
    
    src['text']=src['text'].apply(denoise_text)
    tar['text']=tar['text'].apply(denoise_text)
        
    #---------------------------------------------------------------------------
    
    data1 = utils.flatten_df(src['text'])
    word1 = utils.word_tokenize(data1)
    data2 = utils.flatten_df(tar['text'])
    word2 = utils.word_tokenize(data2)
    
    jsd=utils.jsd_for_word(word1, word2)
    ada=utils.adaptivity_word(src['text'], tar['text'], word1, word2, threshold=0.6)
    l2d=utils.l2d_btw_domains(word1, word2)
    
    #----------------------------------------------------------------------------
    
    x_train_src,x_test_src,y_train_src,y_test_src = train_test_split(src.text,src.label,random_state = 0 , stratify = src.label)
    x_train_tar,x_test_tar,y_train_tar,y_test_tar = train_test_split(tar.text,tar.label,random_state = 0 , stratify = tar.label)

    x_train_src = fast_encode(x_train_src.values, fast_tokenizer, maxlen=400)
    x_test_src = fast_encode(x_test_src.values, fast_tokenizer, maxlen=400)
    
    x_train_tar = fast_encode(x_train_tar.values, fast_tokenizer, maxlen=400)
    x_test_tar = fast_encode(x_test_tar.values, fast_tokenizer, maxlen=400)

    # Model Save
    filename = re.search('input/(.*).csv', src_name).group(1).split('/')[1]+"_trained_model"
    model_src = build_model(bert_model, max_len=400)
    if filename in arr_model_already_trained: # model saved
        print("Already present-",filename)
        model_src.load_weights(filename)

        print("Loaded")
    else: # model not saved
        print("New model-",filename)
        
        model_src.fit(x_train_src,y_train_src,batch_size = 32 ,validation_data=(x_test_src,y_test_src),epochs = 3)
        arr_model_already_trained.append(filename)
        model_src.save_weights(filename)
        print("Weights saved")
        
    src_acc = model_src.evaluate(x_test_src,y_test_src, verbose=1)[1]*100
    print("src_acc-",src_acc)
    
    ser_acc = model_src.evaluate(x_train_tar,y_train_tar, verbose=1)[1]*100
    print("ser_acc-",ser_acc)
    
    model_tar = build_model(bert_model, max_len=400)
    model_tar.fit(x_train_tar,y_train_tar,batch_size = 32, epochs = 3)
    ft_acc = model_src.evaluate(x_test_tar,y_test_tar, verbose=1)[1]*100
    print("ft_acc-",ft_acc)

    return [jsd,ada,l2d,src_acc, ser_acc, ft_acc]

In [10]:
comb = list(itertools.permutations(datasets, 2))
src_list = []
tar_list = []
src_acc_list = []
ser_acc_list = []
ft_acc_list = []
jsd_list = []
ada_list = []
l2d_list = []
for src, tar in tqdm(comb, leave=False):
    jsd, ada, l2d, src_acc, ser_acc, ft_acc = model_serving(src, tar)
    jsd_list.append(jsd)
    ada_list.append(ada)
    src=re.search('input/(.*).csv', src).group(1).split('/')[1]
    tar=re.search('input/(.*).csv', tar).group(1).split('/')[1]
    src_list.append(src)
    tar_list.append(tar)
    l2d_list.append(l2d)
    src_acc_list.append(src_acc)
    ser_acc_list.append(ser_acc)
    ft_acc_list.append(ft_acc)

New model- Movie_1_trained_model
Epoch 1/3
47/47 [==============================] - 42s 885ms/step - loss: 0.7148 - accuracy: 0.5080 - val_loss: 0.6880 - val_accuracy: 0.5220
Epoch 2/3
47/47 [==============================] - 40s 852ms/step - loss: 0.6765 - accuracy: 0.5767 - val_loss: 0.6232 - val_accuracy: 0.7200
Epoch 3/3
47/47 [==============================] - 40s 853ms/step - loss: 0.5701 - accuracy: 0.7120 - val_loss: 0.4576 - val_accuracy: 0.8160
Weights saved
16/16 [==============================] - 4s 237ms/step - loss: 0.4576 - accuracy: 0.8160
src_acc- 81.5999984741211
47/47 [==============================] - 12s 252ms/step - loss: 0.4738 - accuracy: 0.8073
ser_acc- 80.73333501815796
Epoch 1/3
47/47 [==============================] - 36s 767ms/step - loss: 0.4774 - accuracy: 0.7767
Epoch 2/3
47/47 [==============================] - 36s 768ms/step - loss: 0.3227 - accuracy: 0.8727
Epoch 3/3
16/16 [==============================] - 4s 234ms/step - loss: 0.4364 - accuracy: 0.7

/kaggle/working/model_discovery/utils.py:356: RuntimeWarning: invalid value encountered in true_divide
  prob1 = prob1 / np.sum(prob1)


New model- Financial_News_2_trained_model
Epoch 1/3
47/47 [==============================] - 42s 893ms/step - loss: 0.6424 - accuracy: 0.6707 - val_loss: 0.6227 - val_accuracy: 0.6860
Epoch 2/3
47/47 [==============================] - 40s 854ms/step - loss: 0.6229 - accuracy: 0.6853 - val_loss: 0.5862 - val_accuracy: 0.6860
Epoch 3/3
47/47 [==============================] - 40s 854ms/step - loss: 0.3860 - accuracy: 0.8120 - val_loss: 0.1986 - val_accuracy: 0.9160
Weights saved
16/16 [==============================] - 4s 234ms/step - loss: 0.1986 - accuracy: 0.9160
src_acc- 91.60000085830688
47/47 [==============================] - 12s 254ms/step - loss: 0.8784 - accuracy: 0.6487
ser_acc- 64.86666798591614
Epoch 1/3
47/47 [==============================] - 36s 767ms/step - loss: 0.4170 - accuracy: 0.8147
Epoch 2/3
47/47 [==============================] - 36s 769ms/step - loss: 0.2894 - accuracy: 0.8713
Epoch 3/3
16/16 [==============================] - 4s 237ms/step - loss: 0.4732 - acc

/kaggle/working/model_discovery/utils.py:356: RuntimeWarning: invalid value encountered in true_divide
  prob1 = prob1 / np.sum(prob1)


Already present- Financial_News_2_trained_model
Loaded
16/16 [==============================] - 4s 236ms/step - loss: 0.1986 - accuracy: 0.9160
src_acc- 91.60000085830688
47/47 [==============================] - 12s 249ms/step - loss: 0.7803 - accuracy: 0.6700
ser_acc- 67.00000166893005
Epoch 1/3
47/47 [==============================] - 36s 769ms/step - loss: 0.3316 - accuracy: 0.8613
Epoch 2/3
47/47 [==============================] - 36s 768ms/step - loss: 0.2128 - accuracy: 0.9293
Epoch 3/3
16/16 [==============================] - 4s 233ms/step - loss: 0.8396 - accuracy: 0.5340
ft_acc- 53.39999794960022


...

Already present- Financial_News_1_trained_model
Loaded
16/16 [==============================] - 4s 237ms/step - loss: 0.0829 - accuracy: 0.9680
src_acc- 96.79999947547913
47/47 [==============================] - 12s 252ms/step - loss: 1.5833 - accuracy: 0.6113
ser_acc- 61.133331060409546
Epoch 1/3
47/47 [==============================] - 36s 771ms/step - loss: 0.6441 - accuracy: 0.6353
Epoch 2/3
47/47 [==============================] - 36s 769ms/step - loss: 0.5149 - accuracy: 0.7607
Epoch 3/3
16/16 [==============================] - 4s 235ms/step - loss: 0.8500 - accuracy: 0.6840
ft_acc- 68.4000015258789


/kaggle/working/model_discovery/utils.py:356: RuntimeWarning: invalid value encountered in true_divide
  prob1 = prob1 / np.sum(prob1)


Already present- Financial_News_1_trained_model
Loaded
16/16 [==============================] - 4s 234ms/step - loss: 0.0829 - accuracy: 0.9680
src_acc- 96.79999947547913
47/47 [==============================] - 12s 252ms/step - loss: 1.5216 - accuracy: 0.6233
ser_acc- 62.33333349227905
Epoch 1/3
47/47 [==============================] - 36s 770ms/step - loss: 0.6553 - accuracy: 0.6347
Epoch 2/3
47/47 [==============================] - 36s 769ms/step - loss: 0.5398 - accuracy: 0.7380
Epoch 3/3
16/16 [==============================] - 4s 239ms/step - loss: 0.8697 - accuracy: 0.6700
ft_acc- 67.00000166893005


/kaggle/working/model_discovery/utils.py:356: RuntimeWarning: invalid value encountered in true_divide
  prob1 = prob1 / np.sum(prob1)


Already present- Financial_News_1_trained_model
Loaded
16/16 [==============================] - 4s 235ms/step - loss: 0.0829 - accuracy: 0.9680
src_acc- 96.79999947547913
47/47 [==============================] - 12s 251ms/step - loss: 0.5472 - accuracy: 0.8313
ser_acc- 83.1333339214325
Epoch 1/3
47/47 [==============================] - 36s 771ms/step - loss: 0.1732 - accuracy: 0.9367
Epoch 2/3
47/47 [==============================] - 36s 769ms/step - loss: 0.0822 - accuracy: 0.9653
Epoch 3/3
16/16 [==============================] - 4s 236ms/step - loss: 0.2828 - accuracy: 0.9100
ft_acc- 91.00000262260437
New model- Tweets_1_trained_model
Epoch 1/3
47/47 [==============================] - 44s 929ms/step - loss: 0.6656 - accuracy: 0.6093 - val_loss: 0.5962 - val_accuracy: 0.6980
Epoch 2/3
47/47 [==============================] - 40s 855ms/step - loss: 0.5399 - accuracy: 0.7240 - val_loss: 0.5764 - val_accuracy: 0.6920
Epoch 3/3
47/47 [==============================] - 40s 856ms/step - lo

In [11]:
result_df = pd.DataFrame({'Source': src_list, 'Target': tar_list, 'JSD': jsd_list,'Adaptivity': ada_list, 'L2D':l2d_list,
                         'Source Accuracy': src_acc_list, 'Serving Accuracy': ser_acc_list, 'Finetune Accuracy': ft_acc_list})

In [12]:
result_df=result_df.sort_values(by=['Target','Serving Accuracy'],ascending=False)
result_df.reset_index(drop=True, inplace=True)
result_df.to_csv('Sentiment_serving-train.csv', index=False)
result_df

,Source,Target,JSD,Adaptivity,L2D,Source Accuracy,Serving Accuracy,Finetune Accuracy
0,Movie_2,Tweets_hatred_5,0.149241,6.714286,0.089254,80.800003,92.666668,6.400000
1,Financial_News_1,Tweets_hatred_5,0.490986,0.857143,0.159169,96.799999,83.133334,91.000003
2,Movie_1,Tweets_hatred_5,0.147102,6.142857,0.088685,81.599998,77.466667,93.599999
3,Tweets_1,Tweets_hatred_5,0.081495,7.000000,0.053569,69.400001,73.000002,59.399998
4,Financial_News_2,Tweets_hatred_5,0.505265,4.285714,0.163838,91.600001,71.533334,89.999998
5,Tweets_2,Tweets_hatred_5,0.084933,7.000000,0.055481,68.599999,67.199999,52.600002
6,Tweets_1,Tweets_2,0.014739,7.000000,0.019774,69.400001,68.466669,69.000000
7,Financial_News_1,Tweets_2,0.527444,0.000000,0.152935,96.799999,62.333333,67.000002
8,Financial_News_2,Tweets_2,0.542546,3.000000,0.156966,91.600001,60.399997,64.999998
9,Movie_1,Tweets_2,0.131600,5.857143,0.079972,81.599998,60.200000,64.999998


In [13]:
arr_sr= (np.asarray(list(set(result_df['Source'].values))))
dict_arr_src=dict()
for i in range(len(arr_sr)):
    dict_arr_src[arr_sr[i]]=0
dict_arr_src

{'Movie_1': 0,
 'Tweets_1': 0,
 'Financial_News_2': 0,
 'Tweets_hatred_5': 0,
 'Financial_News_1': 0,
 'Movie_2': 0,
 'Tweets_2': 0}

In [14]:
arr_target= np.asarray(list(set(result_df['Target'].values)))
arr_target

array(['Movie_1', 'Tweets_1', 'Financial_News_2', 'Tweets_hatred_5',
       'Financial_News_1', 'Movie_2', 'Tweets_2'], dtype='<U16')

In [15]:
from collections import Counter

In [16]:
datasets

['../input/imbd-data-1/Movie_1.csv',
 '../input/imbd-data-1/Movie_2.csv',
 '../input/financial-news-data-2/Financial_News_2.csv',
 '../input/financial-news-data-2/Financial_News_1.csv',
 '../input/tweets-data-4/Tweets_1.csv',
 '../input/tweets-data-4/Tweets_2.csv',
 '../input/tweets-hatred-data-5/Tweets_hatred_5.csv']

In [17]:
df_target_dic={}
for d in datasets:
    f_name = re.search('input/(.*).csv', d).group(1).split('/')[1]
    df_target_dic[f_name]=d
df_target_dic

{'Movie_1': '../input/imbd-data-1/Movie_1.csv',
 'Movie_2': '../input/imbd-data-1/Movie_2.csv',
 'Financial_News_2': '../input/financial-news-data-2/Financial_News_2.csv',
 'Financial_News_1': '../input/financial-news-data-2/Financial_News_1.csv',
 'Tweets_1': '../input/tweets-data-4/Tweets_1.csv',
 'Tweets_2': '../input/tweets-data-4/Tweets_2.csv',
 'Tweets_hatred_5': '../input/tweets-hatred-data-5/Tweets_hatred_5.csv'}

In [18]:
# metrics=['JSD','Adaptivity','L2D','Source Accuracy']
# for m in metrics:
#     temp_df=[]
#     print("--------")
#     for target in arr_target:
#         temp_df = result_df[result_df['Target']==target][['Source','Target',m]]
#         temp_df= temp_df.nsmallest(3, m)
#         print("TARGET-",target)
#         print("Data-")
#         print(temp_df)
#         for src in temp_df['Source'].values:
#             # Model Save
#             file_name = src+"_trained_model"
#             if file_name in arr_model_already_trained: # model is already saved
#                 model_src = build_model(bert_model, max_len=400)
#                 model_src.load_weights(file_name)
#                 print("")
#                 print("Weights Loaded-      ",file_name)
#             print("Target dataset loaded-    ",target)
#             df_target = pd.read_csv(df_target_dic[target]).sample(20, replace=True,random_state=10)
#             # Each Row
#             for curr_row_tar in df_target['text']:
#                 curr_row_tar=denoise_text(curr_row_tar)
#                 print(curr_row_tar)
#                 curr_row_tar = fast_encode([curr_row_tar], fast_tokenizer, maxlen=400)
#                 print(curr_row_tar)
#                 break
                


    
#             break
#         break
#     break

In [19]:
arr_model_already_trained

['Movie_1_trained_model',
 'Movie_2_trained_model',
 'Financial_News_2_trained_model',
 'Financial_News_1_trained_model',
 'Tweets_1_trained_model',
 'Tweets_2_trained_model',
 'Tweets_hatred_5_trained_model']